# Python Notional Machine

Our goal is to refresh ourselves on basics (and some subtleties) associated with Python's data and computational model. Along the way, we'll also use or refresh ourselves on the <b>environment model</b> as a way to think about and keep track of the effect of executing python code (in particular, keeping track of variable bindings).

## Variables and data types

### Integers

In [1]:
a = 307
b = a
print('a:', a, '\nb:', b)

a: 307 
b: 307


In [2]:
a = a + 310
a += 400
print('a:', a, '\nb:', b)

a: 1017 
b: 307


So far so good -- integers, and variables pointing to integers, are straightforward.

### Lists

In [3]:
x = ['baz', 302, 303, 304]
print('x:', x)

x: ['baz', 302, 303, 304]


In [4]:
y = x
print('y:', y)

y: ['baz', 302, 303, 304]


In [5]:
x = 377
print('x:', x, '\ny:', y)

x: 377 
y: ['baz', 302, 303, 304]



Unlike integers, lists are mutable:

In [6]:
x = y
x[0] = 388
print('x:', x)

x: [388, 302, 303, 304]


In [7]:
print('y:', y)=

y: [388, 302, 303, 304]


As seen above, we have to be careful about sharing (also known as "aliasing") mutable data!

In [15]:
a = [301, 302, 303]
b = [a, a, a]
print(b)

[[301, 302, 303], [301, 302, 303], [301, 302, 303]]


In [17]:
b[0][0] = 304
print(b)

[[304, 302, 303], [304, 302, 303], [304, 302, 303]]


Tuples are a lot like lists, except that they are immutable.

In [18]:
x = ('baz', [301, 302], 303, 304)
y = x
print('x:', x, '\ny:', y)

x: ('baz', [301, 302], 303, 304) 
y: ('baz', [301, 302], 303, 304)


Unlike a list, we can't change the top most structure of a tuple; trying to change it results in an error:

In [19]:
x[0] = 388

TypeError: 'tuple' object does not support item assignment

What will happen in the following (operating on x)?

In [20]:
x[1][0] = 311
print('x:', x, '\ny:', y)

x: ('baz', [311, 302], 303, 304) 
y: ('baz', [311, 302], 303, 304)


So we still need to be careful! The tuple didn't change at the top level -- but it might have members that are themselves mutable.

### Strings

Strings are also immutable. We can't change them once created. 

In [21]:
a = 'ya'
b = a + 'rn'
print('a:', a, '\nb:', b)

a: ya 
b: yarn


In [22]:
a[0] = 'Y'

TypeError: 'str' object does not support item assignment

In [23]:
c = 'twine'
d = c
c += ' thread'
print('c:', c, '\nd:', d)

c: twine thread 
d: twine


That's a little bit tricky. Here the '+=' operator makes a copy of c first to use as part of the new string with ' there' included at the end.

### Back to lists: append, extend, and the '+' and '+=' operators

In [24]:
x = [301, 302, 303]
y = x
x.append([304, 305])
print('x:', x, '\ny:', y)

x: [301, 302, 303, [304, 305]] 
y: [301, 302, 303, [304, 305]]


So again, we have to watch out for aliasing/sharing, whenever we mutate an object.

In [25]:
x = [301, 302, 303]
y = x
x.extend([304, 305])
print('x:', x, '\ny:', y)

x: [301, 302, 303, 304, 305] 
y: [301, 302, 303, 304, 305]


What happens when using the '+' operator used on lists?

In [26]:
x = [301, 302, 303]
y = x
x = x + [304, 305]
print('x:', x)

x: [301, 302, 303, 304, 305]


So the '+' operator on a list looks sort of like extend. But has it changed x in place, or made a copy of x first for use in the longer list?

And what happens to <tt>y</tt> in the above?

In [27]:
print('y:', y)

y: [301, 302, 303]


So that clarifies things -- the "+" operator on a list makes a (shallow) copy of the left argument first, then uses that copy in the new larger list.

## Functions and scoping

In [28]:
x = 500
def foo(y):
    return x + y
z = foo(307)
print('x:', x, '\nfoo:', foo, '\nz:', z)

x: 500 
foo: <function foo at 0x1040827a0> 
z: 807


In [29]:
def bar(x):
    x = 1000
    return foo(307)
w = bar('hi')
print('x:', x, '\nw:', w)

x: 500 
w: 807


Importantly, foo "remembers" that it was created in the global environment, so looks in the global environment to find a value for 'x'. It does NOT look back in its "call chain"; rather, it looks back in its parent environment.

### Optional arguments and default values

In [30]:
def foo(x, y = []):
    y = y + [x]
    return y

a = foo(307)
b = foo(388, [301, 302, 303])
print('a:', a, '\nb:', b)

a: [307] 
b: [301, 302, 303, 388]


In [31]:
c = foo(307)
print('a:', a, '\nb:', b, '\nc:', c)

a: [307] 
b: [301, 302, 303, 388] 
c: [307]


Let's try something that looks close to the same thing... but with an important difference!

In [32]:
def foo(x, y = []):
    y.append(x)   # different here
    return y

a = foo(307)
b = foo(388, [301, 302, 303])
print('a:', a, '\nb:', b)

a: [307] 
b: [301, 302, 303, 388]


Okay, so far it looks the same as with the earlier foo.

In [33]:
c = foo(307)
print('a:', a, '\nb:', b, '\nc:', c)

a: [307, 307] 
b: [301, 302, 303, 388] 
c: [307, 307]


So quite different... all kinds of aliasing going on. The moral here is to be VERY careful (and indeed it may be best to simply avoid) having optional/default arguments that are mutable structures like lists... it's hard to remember or debug such aliasing!

## Closures

In [35]:
def add_n(n):
    def inner(x):
        return x + n
    return inner

In [41]:
add1 = add_n(301)
add2 = add_n(302)

print(add2(303))
print(add1(307)) 
print(add_n(308)(309))  

605
608
617
